In [1]:
!pip install keras==2.1.1

You are using pip version 18.1, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install tensorflow==1.4.0

You are using pip version 18.1, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import pkg_resources
pkg_resources.require('keras==2.1.1')
pkg_resources.require('tensorflow==1.4.0')
import keras

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
import glob
import numpy as np
from music21 import converter, instrument, note, chord, stream
from keras.utils import to_categorical
from keras import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
from keras.callbacks import ModelCheckpoint

In [7]:
notes = []

for file in glob.glob("midi_songs_guitar/*.mid"):
    try:
        midi = converter.parse(file)
    except:
        continue
        
    notes_to_parse = None
    
    parts = instrument.partitionByInstrument(midi)
    
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else:
        notes_to_parse = midi.flat.notes
        
    # Append the pitch of every note
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        # If chord, encode ID of every note in cord separated by .
        elif isinstance(element,chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

In [8]:
# Convert string notes into integers for stronger LSTM performance

# get all pitch names
pitchnames = sorted(set(item for item in notes))

# create a dictionary to map pitches to integers
note_to_int = dict((note,number) for number, note in enumerate(pitchnames))

In [9]:
# create input/output sequences -- output is next note 

sequence_length = 100

network_input = []
network_output = []

# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length,1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
    
network_output = np.array(network_output)
n_patterns = len(network_input)
print(n_patterns, 'training examples')

# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length,1))

print('LSTM Shape:', network_input.shape)

# normalize input
n_vocab = len(note_to_int)
# network_input = network_input / float(n_vocab)

network_output = to_categorical(network_output)
# network_output = np.reshape(network_output, (network_output.shape[0], 1, network_output.shape[1]))

184294 training examples
LSTM Shape: (184294, 100, 1)


In [10]:
network_input[:10]

array([[[410],
        [442],
        [411],
        [428],
        [459],
        [433],
        [460],
        [418],
        [451],
        [441],
        [456],
        [418],
        [443],
        [442],
        [456],
        [418],
        [443],
        [442],
        [250],
        [410],
        [442],
        [411],
        [428],
        [459],
        [418],
        [460],
        [433],
        [451],
        [441],
        [456],
        [418],
        [443],
        [442],
        [456],
        [418],
        [443],
        [442],
        [250],
        [253],
        [408],
        [428],
        [156],
        [443],
        [428],
        [174],
        [331],
        [331],
        [434],
        [418],
        [ 29],
        [285],
        [285],
        [428],
        [411],
        [250],
        [418],
        [443],
        [418],
        [456],
        [442],
        [209],
        [442],
        [442],
        [375],
        [428],
        [434],
        [4

In [11]:
network_output.shape

(184294, 465)

In [13]:
model = Sequential()
model.add(LSTM(
    256,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# model.load_weights('weights/test_weights_v1.hdf5')

In [16]:
model = Sequential()
model.add(LSTM(20,  
               input_shape=(network_input.shape[1], network_input.shape[2]),
               return_sequences=True))
model.add(LSTM(20))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.load_weights("weights/test_weights_simple_guitar_v1.hdf5")

In [47]:
filepath = "weights/test_weights_simple_guitar_v1.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=10, batch_size=256, callbacks=callbacks_list)

Epoch 1/10
184294/184294 [==============================] - 107s 582us/step - loss: 3.4147
Epoch 2/10
184294/184294 [==============================] - 110s 594us/step - loss: 3.4065
Epoch 3/10
184294/184294 [==============================] - 108s 584us/step - loss: 3.4159
Epoch 4/10
184294/184294 [==============================] - 104s 563us/step - loss: 3.4158
Epoch 5/10
184294/184294 [==============================] - 105s 569us/step - loss: 3.4004
Epoch 6/10
184294/184294 [==============================] - 106s 576us/step - loss: 3.3974
Epoch 7/10
184294/184294 [==============================] - 105s 570us/step - loss: 3.4429
Epoch 8/10
184294/184294 [==============================] - 109s 590us/step - loss: 3.3930
Epoch 9/10
184294/184294 [==============================] - 107s 579us/step - loss: 3.3674
Epoch 10/10
184294/184294 [==============================] - 107s 580us/step - loss: 3.3479


In [21]:
# model = Sequential()
# # model.add(LSTM(
# #     256,
# #     input_shape=(network_input.shape[1], network_input.shape[2]),
# #     return_sequences=True
# # ))
# model.add(LSTM(20, return_sequences=True))
# model.add(LSTM(20))
# model.add(Dense(358))
# model.add(Activation('softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
# # Load the weights to each node
# model.load_weights('one_lstm_02_5.4238_bigger.hdf5', by_name=True)

In [48]:
# Pick a random start sequence
start = np.random.randint(0, len(network_input)-1)
pattern = network_input[start]
# pattern = np.load('good_pattern.npy')

# The reverse of note_to_int
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

In [49]:
# for i in range(20):
#     start = np.random.randint(0, len(network_input)-1)
#     pattern = network_input[start]
    
#     # Reshape and scale sequence
#     prediction_input = np.reshape(pattern, (1, len(pattern), 1))
# #     prediction_input = prediction_input / float(n_vocab)
    
#     # Predict next note
#     prediction = model.predict(prediction_input, verbose=0) # Prints probabilities of all note combos
#     index = np.argmax(prediction)
#     result = int_to_note[index]
#     print(result,':',start)

In [50]:
prediction_output = []

# generate 500 notes
for note_index in range(100):
    
    # Reshape and scale sequence
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    
    # Predict next note
    prediction = model.predict(prediction_input, verbose=0) # Prints probabilities of all note combos
    
    # Best Note:
    draw = np.argmax(prediction)

    # Draw from Top N based on weights
#     N = 24
#     indx_twelve = prediction.argsort()[0][-N:]
#     val_twelve = np.partition(prediction.flatten(), -2)[-N:]
#     val_twelve_scaled = val_twelve / sum(val_twelve)
#     draw = np.random.choice(indx_twelve, 1, p=val_twelve_scaled.flatten())[0]
    
    # Add note to our new sequence 
    result = int_to_note[draw]
    prediction_output.append(result)
    
    # Move pattern over 1
    pattern = np.append(pattern,draw)
    pattern = pattern[1:len(pattern)]
    #     print(np.round(prediction,2))
    #     print(draw)
    #     print(index, result)

In [51]:
prediction_output

['C#-1',
 'C#-1',
 'C#-1',
 'C#-1',
 'C#-1',
 'C#-1',
 'C#-1',
 'C#-1',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'C#-1',
 'C#-1',
 'C#-1',
 'C#-1',
 'C#-1',
 'C#-1',
 'C#-1',
 'C#-1',
 'C#-1',
 'C#-1',
 'C#-1',
 'C#-1',
 'E3',
 'C#-1',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'C#-1',
 'C#-1',
 'C#-1',
 'C#-1',
 'E4',
 'C#-1',
 'E4',
 'E4',
 'C#-1',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'C#-1',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'E4',
 'C#-1',
 'E4',
 'C#-1',
 'E4',
 'E4',
 'E4',
 'C#-1',
 'E4']

In [ ]:
import matplotlib.pyplot as plt
plt.hist(prediction.flatten())
plt.show()

In [ ]:
offset = 0
output_notes = []

# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
        
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [ ]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='new_test_garbage_100.mid')

In [ ]:
# prediction_output

In [ ]:
pattern

In [ ]:
# pattern

In [ ]:
# network_input[2]

In [ ]:
# sequence_in

In [ ]:
model.summary()